* Necessary Libraries

In [1]:
import os
import pandas as pd
import numpy as np
from math import sqrt

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


Read in data 

In [2]:
# read text file into pyspark dataframe
#df = spark.read.csv("ds5559/FinalProject/ComboCities.csv",  inferSchema=True, header = True)
df = pd.read_csv("ComboCities.csv")

In [3]:
df.head()

,City,All industry total (in thousands),Private industries,"Agriculture, forestry, fishing and hunting","Mining, quarrying, and oil and gas extraction",Utilities,Construction,Manufacturing,Durable goods manufacturing,Nondurable goods manufacturing,...,Small Market,No Teams,NFL Super Bowl Winner,NFL Division Champion,NFL Playoff Participants,NFL Win Percentage,NFL Wins,NFL Home Attendance,Average Attendance Per Game,Year
0,Atlanta,209741698,191749470,366262,331505,2100718,(D),(D),(D),(D),...,0.0,0.0,0.0,0.0,0.0,0.4375,7.0,425717.0,53214.625,2001.0
1,Austin,55307638,47708348,21993,205947,381802,4324915,7829516,(D),(D),...,0.0,1.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000,2001.0
2,Boise,16435618,14448912,(D),13511,57379,1174578,3196453,(D),(D),...,0.0,1.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000,2001.0
3,Boston,236051306,213403308,(D),87120,2666446,9560212,26949729,(D),(D),...,0.0,0.0,1.0,1.0,1.0,0.6875,11.0,482336.0,60292.000,2001.0
4,Charlotte,76027214,69766921,(D),105937,1805777,3895799,14555666,(D),(D),...,1.0,0.0,0.0,0.0,0.0,0.0625,1.0,579080.0,72385.000,2001.0


Initial review of data

In [4]:
len(df)

432

In [5]:
len(df.columns)

97

In [6]:
df.dtypes

City                                                  object
All industry total (in thousands)                      int64
  Private industries                                   int64
    Agriculture, forestry, fishing and hunting        object
    Mining, quarrying, and oil and gas extraction     object
                                                      ...   
NFL Win Percentage                                   float64
NFL Wins                                             float64
NFL Home Attendance                                  float64
Average Attendance Per Game                          float64
Year                                                 float64
Length: 97, dtype: object

In [7]:
df.columns

Index(['City', 'All industry total (in thousands)', '  Private industries',
       '    Agriculture, forestry, fishing and hunting',
       '    Mining, quarrying, and oil and gas extraction', '    Utilities',
       '    Construction', '    Manufacturing',
       '      Durable goods manufacturing',
       '      Nondurable goods manufacturing', '    Wholesale trade',
       '    Retail trade', '    Transportation and warehousing',
       '    Information',
       '    Finance, insurance, real estate, rental, and leasing',
       '      Finance and insurance',
       '      Real estate and rental and leasing',
       '    Professional and business services',
       '      Professional, scientific, and technical services',
       '      Management of companies and enterprises',
       '      Administrative and support and waste management and remediation services',
       '    Educational services, health care, and social assistance',
       '      Educational services', '      Health 

Formatting Data

In [8]:
#Replace all (D) with NA
#df.replace({'(D)': np.nan }, regex=True)
df = df.replace(to_replace='(D)', value=np.NaN)

In [9]:
#Find percentage of na per column
#total team reven MLB NA for 2002 NOTE 

x=df.isnull().sum().sort_values(ascending=False) / len(df)
#df.isna().sum().sort_values(ascending=False) / len(df)
#x.index[1]
#y=x[x>0.25]
#z=y.index.to_list()
#

In [10]:
#Going to drop any column with 25% or more missing data
#df2=df[z]
df = df.loc[:, x < .25]

In [11]:
#Expected to drop 20 columns, and confirmed via code below
len(df.columns)

77

In [16]:
pd.set_option('display.max_rows',None)
df.dtypes

City                                                                         object
All industry total (in thousands)                                             int64
  Private industries                                                          int64
    Mining, quarrying, and oil and gas extraction                            object
    Construction                                                             object
    Manufacturing                                                            object
    Retail trade                                                             object
    Finance, insurance, real estate, rental, and leasing                      int64
      Finance and insurance                                                  object
      Real estate and rental and leasing                                     object
    Professional and business services                                       object
    Educational services, health care, and social assistance                

In [29]:
df["      Farm proprietors' income"]

0      188311
1        1835
2      108356
3       -9520
4      323816
5       75111
6       39241
7         946
8      -11414
9       46379
10      38818
11      57538
12      41962
13      17176
14      10034
15     -16989
16      18440
17       5447
18      40137
19      18993
20      99423
21      -4158
22        342
23      24169
24      95210
25       6500
26      86936
27      -8215
28     156061
29      32308
30      36498
31     -18538
32     -21711
33      35656
34      59917
35      86560
36       -712
37      16528
38      11065
39     -63642
40       6471
41       6193
42      47136
43      13684
44     100740
45        603
46      -1092
47      25883
48     110707
49      30582
50      65229
51       1899
52     198014
53      56100
54       4265
55     -36672
56     -29113
57      76183
58      94241
59     132733
60      24203
61      35786
62      83840
63     -74825
64      26525
65       8899
66      74051
67      56454
68      82515
69       6705
70       -395
71    

In [30]:
df = df.replace(to_replace='na', value=int(0))
df = df.replace(to_replace=np.NaN, value=int(0))
df = df.replace(to_replace='(L)', value=int(0))

In [36]:
#Changes all columns except the first to numeric. In order to avoid muddling regression results
df.iloc[:,1:] = df.iloc[:,1:].apply(pd.to_numeric)
df.dtypes

City                                                                         object
All industry total (in thousands)                                             int64
  Private industries                                                          int64
    Mining, quarrying, and oil and gas extraction                             int64
    Construction                                                              int64
    Manufacturing                                                             int64
    Retail trade                                                              int64
    Finance, insurance, real estate, rental, and leasing                      int64
      Finance and insurance                                                   int64
      Real estate and rental and leasing                                      int64
    Professional and business services                                        int64
    Educational services, health care, and social assistance                

In [35]:
#testing if negative values are now treated as numerics NOT strings
df["      Farm proprietors' income"].str.isdigit()

AttributeError: Can only use .str accessor with string values!

In [39]:
#Convert categorical columns to type category to make our model functions easier to deal with. 

df[["Large Market", "Medium Market","Small Market","Year","City","No Teams","NFL Super Bowl Winner","NFL Division Champion","NFL Playoff Participants","World Series Title",\
    "Division Title MLB"]] = df[["Large Market", "Medium Market","Small Market","Year","City","No Teams","NFL Super Bowl Winner","NFL Division Champion","NFL Playoff Participants","World Series Title",\
    "Division Title MLB"]].astype('category')

df.dtypes

City                                                                        category
All industry total (in thousands)                                              int64
  Private industries                                                           int64
    Mining, quarrying, and oil and gas extraction                              int64
    Construction                                                               int64
    Manufacturing                                                              int64
    Retail trade                                                               int64
    Finance, insurance, real estate, rental, and leasing                       int64
      Finance and insurance                                                    int64
      Real estate and rental and leasing                                       int64
    Professional and business services                                         int64
    Educational services, health care, and social assistance     

In [ ]:
#Could potentially replace nans with column averages or median. IF TIME PERMITS 

END PREPROCESSING

In [ ]:
#Begin Model creation 
#NEED To DEFINE Economic impact

#That there is a higher GDP or a lower unemployment in a city with a sprots team versus a city without a sports team when taking market size in to consideration. 

#Do we a have a city before and after a team joined: A couple: Houston Football, DC Baseball, Seattle lost NBA team, LA football gained two football teams

Questions to answer

# 1) Does a city having a sports team affect its GDP or unemployment. Regression 

# 2) Does market size matter to economic impact?

# 3) The year the team joins a league or when ownership changes?


